In [1]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 22.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
from sklearn.metrics import f1_score
import numpy as np
import torch


label_dict= {'neutral': 0, 'surprise': 1, 'fear': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'anger': 6}
def f1_score_func(preds,labels):
    preds_flat=np.argmax(preds,axis=1).flatten()
    labels_flat=labels.flatten()
    return f1_score(labels_flat,preds_flat,average="weighted")

def accuracy_per_class(preds,labels):
    label_dict_inverse={v:k for k, v in label_dict.items()}
    preds_flat=np.argmax(preds,axis=1).flatten()
    labels_flat=labels.flatten()
    totalacc=0
    tot=0
    for label in np.unique(labels_flat):
        y_preds=preds_flat[labels_flat==label]
        y_true=labels_flat[labels_flat==label]
        totalacc+=len(y_preds[y_preds == label])
        tot+=len(y_true)
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds == label])}/{len(y_true)}\n')
    print("Acc=",totalacc/tot)

In [3]:
import random
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer
from transformers import BertTokenizer, BertModel
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
import torch
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
#from evaluate import f1_score_func,accuracy_per_class

In [11]:
seed_val = 994
device=torch.device("cuda")
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
batch_size=16
epochs=5

In [12]:
train_data=pd.read_csv("/content/train_sent_emo.csv")
dev_data=pd.read_csv("/content/dev_sent_emo.csv")
test_data=pd.read_csv("/content/test_sent_emo.csv")

utterances_train=train_data["Utterance"]
utterances_dev=dev_data["Utterance"]
utterances_test=test_data["Utterance"]

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

processed_data_train=[]
processed_data_dev=[]
processed_data_test=[]
# labelsT=train_data["Emotion"]
# labelsD=dev_data["Emotion"]
# labels_train=[]
# labels_dev=[]
possible_label=train_data.Emotion.unique()
label_dict={}
for index,possible_label in enumerate(possible_label):
    label_dict[possible_label]=index

train_data['Emotion'] = train_data.Emotion.replace(label_dict)
dev_data["Emotion"]= dev_data.Emotion.replace(label_dict)
test_data["Emotion"]=test_data.Emotion.replace(label_dict)
# print(train_data.Emotion.values)
# processed_data_train1=[]
# processing data
# for i in utterances_train:
#     # print(i)
#     processed_data_train.append(i)
processed_data_train=utterances_train.values
processed_data_dev=utterances_dev.values
processed_data_test=utterances_test.values

# print(type(processed_data_train))
# print(type(processed_data_train1))
# for i in utterances_dev:
#     processed_data_dev.append(i)
# print(processed_data[1])

In [13]:
# # tokenizing data
encoded_inputs_train=tokenizer.batch_encode_plus(processed_data_train,add_special_tokens=True,
    return_attention_mask=True,
    padding=True,
    max_length=50,
    truncation=True,
    return_tensors='pt')

encoded_inputs_dev=tokenizer.batch_encode_plus(processed_data_dev,add_special_tokens=True,
    return_attention_mask=True,
    padding=True,
    max_length=50,
    truncation=True,
    return_tensors='pt')

encoded_inputs_test=tokenizer.batch_encode_plus(processed_data_test,add_special_tokens=True,
    return_attention_mask=True,
    padding=True,
    max_length=50,
    truncation=True,
    return_tensors='pt')


input_id_train=encoded_inputs_train["input_ids"]
attention_mask_train=encoded_inputs_train["attention_mask"]
labels_train=torch.tensor(train_data.Emotion.values)
# print(input_id_train)
#
input_id_dev=encoded_inputs_dev["input_ids"]
attention_mask_dev=encoded_inputs_dev["attention_mask"]
labels_dev=torch.tensor(dev_data.Emotion.values)


input_id_test=encoded_inputs_test["input_ids"]
attention_mask_test=encoded_inputs_test["attention_mask"]
labels_test=torch.tensor(test_data.Emotion.values)

In [16]:
#
dataset_train=TensorDataset(input_id_train,attention_mask_train,labels_train)
dataset_dev=TensorDataset(input_id_dev,attention_mask_dev,labels_dev)
dataset_test=TensorDataset(input_id_test,attention_mask_test,labels_test)

model=BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=7,
                                                    output_attentions=False,
                                                    output_hidden_states=False)

#dataset_train=dataset_train.to("cuda")
#dataset_dev=dataset_dev.to("cuda")
# Remove or comment out the following line:
device=torch.device("cuda")

# Replace the following line:
model=model.to("cuda")

kwargs = {'num_workers': 1, 'pin_memory': True}
dataloader_train=DataLoader(dataset_train,
                            sampler=RandomSampler(dataset_train),
                            batch_size=batch_size,
                            **kwargs)
dataloader_dev=DataLoader(dataset_dev,
                          sampler=SequentialSampler(dataset_dev),
                          batch_size=batch_size,
                          **kwargs)
dataloader_test=DataLoader(dataset_test,
                          sampler=SequentialSampler(dataset_test),
                          batch_size=len(test_data),
                          **kwargs)


optimizer=AdamW(model.parameters(),
                lr=5e-05,
                eps=1e-08)

scheduler=get_linear_schedule_with_warmup(optimizer,
                                          num_warmup_steps=(0.1*(len(dataloader_train)*epochs)),
                                          num_training_steps=len(dataloader_train)*epochs)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
def evaluate(dataloader_val):
    model.eval()
    loss_val_total=0
    predictions,true_vals=[],[]

    for batch in dataloader_val:
        batch=tuple(b.to(device) for b in batch)

        inputs={"input_ids":batch[0],
                "attention_mask": batch[1],
                "labels":batch[2]}


        with torch.no_grad():
            outputs=model(**inputs)

        loss=outputs[0]
        logits=outputs[1]
        loss_val_total+=loss.item()

        logits=logits.detach().cpu().numpy()
        label_ids=inputs["labels"].cpu().numpy()

        predictions.append(logits)
        true_vals.append(label_ids)
        loss_val_avg=loss_val_total/len(dataloader_val)
        predictions=np.concatenate(predictions,axis=0)
        true_vals=np.concatenate(true_vals,axis=0)

        return loss_val_avg,predictions,true_vals

In [ ]:
# Train
for epoch in tqdm(range(1, epochs + 1)):
    model.train()
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc="Epoch{:1d}".format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)

        inputs = {"input_ids": batch[0],
                  "attention_mask": batch[1],
                  "labels": batch[2]}
        outputs = model(**inputs)
        loss = outputs[0]

        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        progress_bar.set_postfix({"training loss": "{:0.2f}".format(loss.item() / len(batch))})

    torch.save(model.state_dict(), f"./finetuned_epoch{epoch}.model")
    tqdm.write(f"Epoch {epoch}")
    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write(f"Training loss: {loss_train_avg}")

    val_loss, predictions, true_vals = evaluate(dataloader_dev)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

Epoch1:   0%|          | 0/625 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

Epoch1: 100%|██████████| 625/625 [02:01<00:00,  6.33it/s, training loss=0.45]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

  0%|          | 0/5 [02:07<?, ?it/s]

Epoch 1
Training loss: 1.2689579491615295


 20%|██        | 1/5 [02:07<08:30, 127.53s/it]

Validation loss: 0.012063067299979073
F1 Score (Weighted): 0.7125



Epoch2:   0%|          | 0/625 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

 20%|██        | 1/5 [03:59<08:30, 127.53s/it]

...


Epoch4:   0%|          | 0/625 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

 60%|██████    | 3/5 [07:53<03:55, 117.64s/it]

Epoch 4
Training loss: 0.35891377759426835


 80%|████████  | 4/5 [07:54<01:57, 117.61s/it]

Validation loss: 0.015740784576960973
F1 Score (Weighted): 0.667239010989011



Epoch5:   0%|          | 0/625 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

Epoch5:  41%|████      | 257/625 [00:46<01:05,  5.65it/s, training loss=0.02]

In [19]:
# test
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(torch.device('cuda'))

model.load_state_dict(torch.load('finetuned_epoch5.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_test)
accuracy_per_class(predictions, true_vals)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Class: neutral
Accuracy: 973/1256

Class: surprise
Accuracy: 164/281

Class: fear
Accuracy: 10/50

Class: sadness
Accuracy: 59/208

Class: joy
Accuracy: 248/402

Class: disgust
Accuracy: 19/68

Class: anger
Accuracy: 139/345

Acc= 0.617624521072797


In [20]:
print(len(encoded_inputs_train["attention_mask"]))

9989


In [28]:
import torch
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Function to evaluate model performance
def evaluate_model(model, dataloader):
    # Set model to evaluation mode
    model.eval()

    # Initialize lists to store true labels and predicted labels
    true_labels = []
    predicted_labels = []

    # Iterate over batches
    for batch in dataloader:
        # Extract inputs and labels from batch
        input_ids, attention_mask, labels = batch

        # Move inputs and labels to device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # Forward pass
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        # Compute predicted labels
        _, predicted = torch.max(logits, dim=1)

        # Append true labels and predicted labels
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

    # Compute accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)

    # Generate classification report
    class_report = classification_report(true_labels, predicted_labels)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    return accuracy, class_report, conf_matrix

# Evaluate the model on the test set
accuracy, class_report, conf_matrix = evaluate_model(model, dataloader_test)

# Print the performance metrics
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(class_report)
print("Confusion Matrix:")
print(conf_matrix)


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Accuracy: 0.617624521072797
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.77      0.76      1256
           1       0.56      0.58      0.57       281
           2       0.17      0.20      0.18        50
           3       0.34      0.28      0.31       208
           4       0.53      0.62      0.57       402
           5       0.41      0.28      0.33        68
           6       0.49      0.40      0.44       345

    accuracy                           0.62      2610
   macro avg       0.47      0.45      0.45      2610
weighted avg       0.61      0.62      0.61      2610

Confusion Matrix:
[[973  44  21  61 103   8  46]
 [ 39 164   4   6  34   4  30]
 [ 17   5  10   5   6   0   7]
 [ 77   9  10  59  22   3  28]
 [ 91  21   4  12 248   1  25]
 [ 22   5   4   7   4  19   7]
 [ 77  43   6  22  47  11 139]]


In [29]:
import torch
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Function to evaluate model performance
def evaluate_model(model, dataloader):
    # Set model to evaluation mode
    model.eval()

    # Initialize lists to store true labels and predicted labels
    true_labels = []
    predicted_labels = []

    # Define emotion labels
    emotion_labels = ['neutral', 'surprise', 'fear', 'sadness', 'joy', 'disgust', 'anger']

    # Iterate over batches
    for batch in dataloader:
        # Extract inputs and labels from batch
        input_ids, attention_mask, labels = batch

        # Move inputs and labels to device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # Forward pass
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        # Compute predicted labels
        _, predicted = torch.max(logits, dim=1)

        # Append true labels and predicted labels
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

    # Compute accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)

    # Generate classification report with emotion labels
    class_report = classification_report(true_labels, predicted_labels, target_names=emotion_labels)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(true_labels, predicted_labels)

    return accuracy, class_report, conf_matrix

# Evaluate the model on the test set
accuracy, class_report, conf_matrix = evaluate_model(model, dataloader_test)

# Print the performance metrics
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(class_report)
print("Confusion Matrix:")
print(conf_matrix)


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Accuracy: 0.617624521072797
Classification Report:
              precision    recall  f1-score   support

     neutral       0.75      0.77      0.76      1256
    surprise       0.56      0.58      0.57       281
        fear       0.17      0.20      0.18        50
     sadness       0.34      0.28      0.31       208
         joy       0.53      0.62      0.57       402
     disgust       0.41      0.28      0.33        68
       anger       0.49      0.40      0.44       345

    accuracy                           0.62      2610
   macro avg       0.47      0.45      0.45      2610
weighted avg       0.61      0.62      0.61      2610

Confusion Matrix:
[[973  44  21  61 103   8  46]
 [ 39 164   4   6  34   4  30]
 [ 17   5  10   5   6   0   7]
 [ 77   9  10  59  22   3  28]
 [ 91  21   4  12 248   1  25]
 [ 22   5   4   7   4  19   7]
 [ 77  43   6  22  47  11 139]]


In [21]:
# User input sentence
user_sentence = "I wish i was intelligent enough to do this project on my own."

# Tokenize the input sentence
encoded_user_input = tokenizer.encode_plus(
    user_sentence,
    add_special_tokens=True,
    max_length=50,
    truncation=True,
    padding='max_length',
    return_tensors='pt'
)

# Move input tensors to the appropriate device (e.g., CUDA if available)
input_ids = encoded_user_input['input_ids'].to(torch.device('cuda'))
attention_mask = encoded_user_input['attention_mask'].to(torch.device('cuda'))

# Pass the input tensors through the model to obtain predictions
with torch.no_grad():
    outputs = model(input_ids, attention_mask)

# Extract predicted class probabilities
probabilities = torch.softmax(outputs.logits, dim=1).squeeze()

# Determine the predicted class
predicted_class = torch.argmax(probabilities).item()

# Map the predicted class index to the corresponding emotion label
predicted_emotion = list(label_dict.keys())[list(label_dict.values()).index(predicted_class)]

# Print the predicted emotion
print("Predicted emotion:", predicted_emotion)


Predicted emotion: neutral


In [23]:
from sklearn.metrics import classification_report, confusion_matrix

# List of sentences
sentences = [
    "I feel betrayed, I am gonna do this project alone.",
    "This is really weird person.",
    "Damn! Them weirdos always creating fuss!",
    "I am overjoyed to tell you that I got a job.",
    "Hi! You look so handsome today!",
    "I feel so great after exams."
]

# Initialize lists to store predictions and scores
predicted_emotions = []
predicted_scores = []

# Process each sentence
for sentence in sentences:
    # Tokenize the input sentence
    encoded_input = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=50,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )

    # Move input tensors to the appropriate device (e.g., CUDA if available)
    input_ids = encoded_input['input_ids'].to(torch.device('cuda'))
    attention_mask = encoded_input['attention_mask'].to(torch.device('cuda'))

    # Pass the input tensors through the model to obtain predictions
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)

    # Extract predicted class probabilities
    probabilities = torch.softmax(outputs.logits, dim=1).squeeze()

    # Determine the predicted class index and corresponding emotion label
    predicted_class = torch.argmax(probabilities).item()
    predicted_emotion = list(label_dict.keys())[list(label_dict.values()).index(predicted_class)]

    # Store predicted emotion and score
    predicted_emotions.append(predicted_emotion)
    predicted_scores.append(probabilities[predicted_class].item())

# Print predicted emotions and scores for each sentence
for i, (sentence, emotion, score) in enumerate(zip(sentences, predicted_emotions, predicted_scores), 1):
    print(f"Sentence {i}: {sentence}")
    print(f"Predicted Emotion: {emotion} (Score: {score:.4f})")
    print()


Sentence 1: I feel betrayed, I am gonna do this project alone.
Predicted Emotion: sadness (Score: 0.9928)

Sentence 2: This is really weird person.
Predicted Emotion: surprise (Score: 0.7410)

Sentence 3: Damn! Them weirdos always creating fuss!
Predicted Emotion: anger (Score: 0.5534)

Sentence 4: I am overjoyed to tell you that I got a job.
Predicted Emotion: joy (Score: 0.9939)

Sentence 5: Hi! You look so handsome today!
Predicted Emotion: joy (Score: 0.9935)

Sentence 6: I feel so great after exams.
Predicted Emotion: joy (Score: 0.9878)

